In [1]:
import numpy as np
import plotting as pm
import os

from vehicle_reconstruction import vehicle, car_models, vehicle_reconstructor

CAR_MODEL_DIR = "F:\\ApolloScape\\3d_car_instance_sample\\3d_car_instance_sample\\car_models_json"

In [2]:
vehicles: list[vehicle] = vehicle.load_car_models(car_model_dir=CAR_MODEL_DIR,
                                models=car_models.models)

for id, vehi in enumerate(vehicles):
    print(f"\t{id}:\t{vehi.name:25}", end='\t')
    if id % 4 == 0:
        print()

loading 79 car models
	0:	baojun-310-2017          	
	1:	biaozhi-3008             		2:	biaozhi-liangxiang       		3:	bieke-yinglang-XT        		4:	biyadi-2x-F0             	
	5:	changanbenben            		6:	dongfeng-DS5             		7:	feiyate                  		8:	fengtian-liangxiang      	
	9:	fengtian-MPV             		10:	jilixiongmao-2015        		11:	lingmu-aotuo-2009        		12:	lingmu-swift             	
	13:	lingmu-SX4-2012          		14:	sikeda-jingrui           		15:	fengtian-weichi-2006     		16:	037-CAR02                	
	17:	aodi-a6                  		18:	baoma-330                		19:	baoma-530                		20:	baoshijie-paoche         	
	21:	bentian-fengfan          		22:	biaozhi-408              		23:	biaozhi-508              		24:	bieke-kaiyue             	
	25:	fute                     		26:	haima-3                  		27:	kaidilake-CTS            		28:	leikesasi                	
	29:	mazida-6-2015            		30:	MG-GT-2015               		31:	oubao         

In [3]:
if not os.path.exists(".\\data\\vehicles_original"):
    os.mkdir(".\\data\\vehicles_original")

for vehi in vehicles:
    pm.create_figure_for_inspection(
        [vehi.vertices[:, [0,2,1]]], [vehi.faces], f".\\data\\vehicles_original\\{vehi.name}.html")

In [4]:
idx = 60
print(vehicles[idx])
vehicles[idx].to_trimesh().show()

vehicle("jipu-3")


In [5]:
global_res = 0.1
standard_bbox = [2.5, 1.9, 5.6, -1.25, 1.25, 0.0, 1.90, -2.8, 2.8]
grid_res = [int(np.ceil(standard_bbox[0]/global_res) + 1),
            int(np.ceil(standard_bbox[1]/global_res) + 1),
            int(np.ceil(standard_bbox[2]/global_res) + 1)]
print(grid_res)
print(vehicles[idx])
vehicles[idx].to_voxel(sampling_space=standard_bbox, 
                       grid_res=grid_res).to_trimesh().show()

[26, 20, 57]
vehicle("jipu-3")


In [6]:
vehi_reconstructor = vehicle_reconstructor(vehicles=vehicles, 
                      sampling_space=standard_bbox, 
                      grid_res=grid_res)

In [7]:
use_preprocessed_voxel = True

if use_preprocessed_voxel:
    vehi_reconstructor.load_voxel(".\\preprocessed_voxel.bin")
else:
    vehi_reconstructor.voxelize_vehicles()
    vehi_reconstructor.save_voxel(".\\preprocessed_voxel.bin")

In [8]:
vehi_reconstructor.fit_model(k=5)

In [9]:
reconstructed_vehicles = vehi_reconstructor.reconsturct()

In [10]:
reconstructed_vehicles[idx].to_trimesh().show()

In [11]:
if not os.path.exists(".\\data\\vehicles_reconstructed"):
    os.mkdir(".\\data\\vehicles_reconstructed")

for idx, vehi in enumerate(reconstructed_vehicles):
    vehi = vehi.to_trimesh()
    pm.create_figure_for_inspection(
        [vehi.vertices[:, [0, 2, 1]]], [vehi.faces], f".\\data\\vehicles_reconstructed\\{vehicles[idx].name}.html")

In [14]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider

global vehi
vehi = None

def slidar_create(min=-10.0, max=10.0, desc="desc:"):
    return widgets.FloatSlider(
        value=0.0,
        min=min,
        max=max,
        step=0.1,
        description=desc,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )


def generate_model(a, b, c, d, e):
    latent = torch.Tensor([[a, b, c, d, e]]).cuda()
    global vehi
    vehi = vehi_reconstructor.decode(latent)[0].to_trimesh()
    # pm.create_figure_for_inspection(
    #     [vehi.vertices[:, [0, 2, 1]]], [vehi.faces], f"temp.html")

interact(generate_model, a=slidar_create(-40.0, 40.0, 'v'),
         b=slidar_create(-20.0, 20.0, 'w'),
         c=slidar_create(-20.0, 20.0, 'x'),
         d=slidar_create(-20.0, 20.0, 'y'),
         e=slidar_create(-20.0, 20.0, 'z'))

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='v', max=40.0, min=-40.0, re…

<function __main__.generate_model(a, b, c, d, e)>

In [28]:
vehi.show()